# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [46]:
!pip3 install Pillow
!pip install scipy==1.2.2
!pip uninstall imgaug
!pip install imgaug==0.2.5

Uninstalling imgaug-0.2.5:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/imgaug-0.2.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/imgaug/*
Proceed (y/n)? y
  Successfully uninstalled imgaug-0.2.5
Processing /root/.cache/pip/wheels/31/48/c8/ca3345e8582a078de94243996e148377ef66fdb845557bae0b/imgaug-0.2.5-cp37-none-any.whl


In [47]:
import numpy as np
import os
import datetime
from scipy.misc import imresize
import imageio
from PIL import Image       
import numpy

We set the random seed so that the results don't vary drastically.

In [48]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [49]:
from google.colab import drive
drive.mount('/content/gdrive')
#drive.flush_and_unmount()


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [50]:
root_path='/content/gdrive/MyDrive/Upgrade/gesture_case_study/Project_data'

In [51]:
train_doc = np.random.permutation(open(root_path+'/train.csv').readlines())
val_doc = np.random.permutation(open(root_path+'/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [52]:
len(train_doc)

663

In [53]:
len(val_doc)

100

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [54]:
x = 10 # x is the number of image we want to use in each video
y = 120 # size of image
z = 120 # size of image

In [75]:
def generator(source_path, folder_list, batch_size):
   # print( 'Source path = ', source_path, '; batch size =', batch_size)
   # print('folder_list =', len(folder_list))
    img_idx = [x for x in range(0,batch_size)]#create a list of image numbers you want to use for a particular video
   # print("img_idx: "+str(img_idx))
    while True:
        t = np.random.permutation(folder_list)#select folder from folder list in random suffling basis
        num_batches =len(folder_list)//batch_size # calculate the number of batches
    #    print("num_batches: "+num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
     #           print('num_batches '+str(num_batches)+'folder '+str(folder)+'imgs '+str(imgs))
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #tempImage=imresize(image,(y,z))
                    tempImage=numpy.array(Image.fromarray(image.astype(numpy.uint8)).resize(size=(120, 120)))
                    tempImage=tempImage/127.5-1 #Normalize Data

                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches

        if(len(folder_list)!=num_batches*batch_size):
      #    print("Batch: ",num_batches+1,"Index:", batch_size)
          batch_size = len(folder_list)-num_batches*batch_size
          batch_data = np.zeros(batch_size,x,y,z,3)# x is the number of images you use for each video, (y,z)
          # is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
          for folder in range(batch_size): # iterate over the batch size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
        
                    ####crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #tempImage=imresize(image,(y,z))
                    tempImage=numpy.array(Image.fromarray(image.astype(numpy.uint8)).resize(size=(120, 120)))
                    tempImage=tempImage/127.5-1 #Normalize Data

                    batch_data[folder,idx,:,:,0] = (tempImage[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (tempImage[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (tempImage[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [76]:
curr_dt_time = datetime.datetime.now()
train_path = root_path+'/train'
val_path = root_path+'/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 1


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [77]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

#write your model here
model=Sequential()

model.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(32, #Number of filters 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(64, #Number pf filters 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [83]:
optimiser = optimizers.Adam(learning_rate=0.001)#write your optimizer
model.compile(optimizer=optimiser, loss='sparse_categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_16 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_16 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_17 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_17 (Activation)   (None, 15, 60, 60, 16)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [84]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [85]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [86]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [87]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


InvalidArgumentError: ignored